In [45]:
import subprocess
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean
from geopy.distance import geodesic
import datetime
import netCDF4 
import glob
from geopy.distance import geodesic

In [46]:
tag = 'ft22-879-18'
sat_path = f"/Users/kat/Desktop/Brain/Area/Research/Siegelman_Lab/Data/cleaned_datasets/{tag}_stlite_ds.nc"
seal_path = f"/Users/kat/Desktop/Brain/Area/Research/Siegelman_Lab/Data/cleaned_datasets/{tag}_ds_tag.nc"
stlite_ds = xr.open_dataset(sat_path)
ds_tag = xr.open_dataset(seal_path)

In [47]:
ds_tag

<xarray.Dataset> Size: 166MB
Dimensions:       (depth: 486, time: 4265)
Coordinates:
  * time          (time) datetime64[ns] 34kB 2018-10-21T11:04:20.000011 ... 2...
  * depth         (depth) int64 4kB 15 16 17 18 19 20 ... 496 497 498 499 500
    lat           (time) float64 34kB ...
    lon           (time) float64 34kB ...
    name          (time) <U11 188kB ...
Data variables: (12/18)
    temperature   (depth, time) float64 17MB ...
    salinity      (depth, time) float64 17MB ...
    sla           (time) float64 34kB ...
    eke           (time) float64 34kB ...
    ke_neuro      (time) float64 34kB ...
    sla_neuro     (time) float64 34kB ...
    ...            ...
    N2            (depth, time) float64 17MB ...
    MLD           (time) float64 34kB ...
    bx            (depth, time) float64 17MB ...
    M4            (depth, time) float64 17MB ...
    qErtel        (depth, time) float64 17MB ...
    inv_Ri        (depth, time) float64 17MB ...

In [48]:
stlite_ds

<xarray.Dataset> Size: 116MB
Dimensions:                 (latitude: 161, longitude: 251, time: 80)
Coordinates:
  * time                    (time) datetime64[ns] 640B 2018-10-20 ... 2019-01-07
  * longitude               (longitude) float64 2kB 67.0 67.1 67.2 ... 91.9 92.0
  * latitude                (latitude) float64 1kB -59.0 -58.9 ... -43.1 -43.0
    Lambert_Azimuthal_Grid  int32 4B ...
Data variables:
    sla                     (latitude, longitude, time) float32 13MB ...
    zeta                    (latitude, longitude, time) float32 13MB ...
    ugos                    (latitude, longitude, time) float32 13MB ...
    vgos                    (latitude, longitude, time) float32 13MB ...
    ss                      (latitude, longitude, time) float32 13MB ...
    sn                      (latitude, longitude, time) float32 13MB ...
    EKE                     (latitude, longitude, time) float32 13MB ...
    STRAIN                  (latitude, longitude, time) float32 13MB ...
    zeta_f                  (latitude, longitude, time) float32 13MB ...
Attributes: (12/43)
    title:                               Daily NeurOST (SSH-SST) Sea Surface ...
    summary:                             A mapped level 4 sea surface height ...
    Conventions:                         CF-1.10
    standard_name_vocabulary:            NetCDF Climate and Forecast (CF) Met...
    keywords:                            Oceans > Sea Surface Topography > Se...
    keywords_vocabulary:                 NASA Global Change Master Directory ...
    ...                                  ...
    build_dmrpp_metadata.build_dmrpp:    3.21.0-46
    build_dmrpp_metadata.bes:            3.21.0-46
    build_dmrpp_metadata.libdap:         libdap-3.21.0-27
    build_dmrpp_metadata.configuration:  \n# TheBESKeys::get_as_config()\nAll...
    build_dmrpp_metadata.invocation:     build_dmrpp -c /tmp/bes_conf_Embh -f...
    history_json:                        [{"$schema":"https:\/\/harmony.earth...

In [49]:
def calculate_cumulative_distance(latitudes, longitudes):
    cumulative_distances = [0]  # Start with 0 for the first point
    for i in range(1, len(latitudes)):
        # Calculate the distance between two consecutive points
        distance = geodesic((latitudes[i-1], longitudes[i-1]), (latitudes[i], longitudes[i])).km
        cumulative_distances.append(cumulative_distances[-1] + distance)
    return np.array(cumulative_distances)

seal_lat = ds_tag['lat']
seal_lon = ds_tag['lon']
seal_cumulative_dist = calculate_cumulative_distance(seal_lat, seal_lon)

# Calculate the cumulative distance for the seal's trajectory

In [50]:
seal_cumulative_dist

array([0.00000000e+00, 1.00000002e+00, 2.00000012e+00, ...,
       4.25998844e+03, 4.26098844e+03, 4.26198841e+03], shape=(4265,))

In [55]:
## testing
for i,time in enumerate(ds_tag['time']):
    seal_lat = ds_tag['lat'].isel(time=slice(0, i+1)).values  # Extract latitude up to current step
    seal_lon = ds_tag['lon'].isel(time=slice(0, i+1)).values  # Extract longitude up to current step

len(ds_tag['time'])

4265

In [51]:
!mkdir -p build8
for i in range(0, len(ds_tag['time']), 2):  # Loop with step size of 15
    # Extract the seal's latitude and longitude up to the current time step
    seal_lat = ds_tag['lat'].isel(time=slice(0, i+1)).values  # Extract latitude up to current step
    seal_lon = ds_tag['lon'].isel(time=slice(0, i+1)).values  # Extract longitude up to current step

    # Find the closest time step in the dataset for the seal's current time step
    #find the distance between the satelite time value and the ds_tag time value
    dist_to_time = np.abs(stlite_ds['time'].values - ds_tag['time'].isel(time=i).values)
    closest_time_idx = np.argmin(dist_to_time)  # Index of closest time step

    # Extract the date for the closest time step
    date_str = str(stlite_ds.time.isel(time=closest_time_idx).values.astype('datetime64[D]'))
    time_step = round(seal_cumulative_dist[i])


    # Plot SLA, Zeta, STRAIN/f, and EKE in a 2x2 grid
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 15))

    # Plot SLA
    sla_plot = ax1.pcolormesh(
        stlite_ds.longitude,
        stlite_ds.latitude,
        stlite_ds.sla.isel(time=closest_time_idx),
        vmin=-0.5,
        vmax=0.5,
        cmap="cmo.balance",  
        shading="auto"
    )
    cbar1 = plt.colorbar(sla_plot, ax=ax1, label="Sea Level Anomaly (m)")
    ax1.set_xlabel("Longitude")
    ax1.set_ylabel("Latitude")
    ax1.set_title(f"SLA at: {time_step} m")

    # Plot the seal's trajectory on SLA
    scatter1 = ax1.scatter(
        seal_lon, seal_lat, c=np.arange(len(seal_lon)), cmap="jet", s=10
    )

    # Plot Zeta
    zeta_plot = ax2.pcolormesh(
        stlite_ds.longitude,
        stlite_ds.latitude,
        stlite_ds.zeta_f.isel(time=closest_time_idx),
        vmin=-0.4,
        vmax=0.4,
        cmap="cmo.delta",  
        shading="auto"
    )
    cbar2 = plt.colorbar(zeta_plot, ax=ax2, label="Zeta/f")
    ax2.set_xlabel("Longitude")
    ax2.set_ylabel("Latitude")
    ax2.set_title(f"Zeta at: {time_step} m")

    # Plot the seal's trajectory on Zeta
    scatter2 = ax2.scatter(
        seal_lon, seal_lat, c=np.arange(len(seal_lon)), cmap="jet", s=10
    )

    # Plot STRAIN/f
    strain_plot = ax3.pcolormesh(
        stlite_ds.longitude,
        stlite_ds.latitude,
        stlite_ds.STRAIN.isel(time=closest_time_idx),
        vmin=0,
        vmax=0.4,
        cmap="cmo.deep",  
        shading="auto"
    )
    cbar3 = plt.colorbar(strain_plot, ax=ax3, label="Strain/f")
    ax3.set_xlabel("Longitude")
    ax3.set_ylabel("Latitude")
    ax3.set_title(f"Strain at: {time_step} m")

    # Plot the seal's trajectory on STRAIN/f
    scatter3 = ax3.scatter(
        seal_lon, seal_lat, c=np.arange(len(seal_lon)), cmap="jet", s=10
    )

    # Plot EKE
    EKE_plot = ax4.pcolormesh(
        stlite_ds.longitude,
        stlite_ds.latitude,
        stlite_ds.EKE.isel(time=closest_time_idx),
        vmin=0,
        vmax=1,
        cmap="cmo.speed", 
        shading="auto"
    )
    cbar4 = plt.colorbar(EKE_plot, ax=ax4, label=r"EKE ($m^2/s^2$)")
    ax4.set_xlabel("Longitude")
    ax4.set_ylabel("Latitude")
    ax4.set_title(f"EKE at: {time_step} m")

    # Plot the seal's trajectory on EKE
    scatter4 = ax4.scatter(
        seal_lon, seal_lat, c=np.arange(len(seal_lon)), cmap="jet", s=10
    )

    # Set limits and remove colorbar for the trajectory
    # for ax in [ax1, ax2, ax3, ax4]:
    #     ax.set_xlim(np.nanmin(stlite_ds.longitude) - 3, np.nanmax(stlite_ds.longitude) + 3)
    #     ax.set_ylim(np.nanmin(stlite_ds.latitude) - 3, np.nanmax(stlite_ds.latitude) + 3)
    
    fig.suptitle(f"Trajectory of Tag: {tag}", fontsize=20, fontweight="bold")
    # Save the frame
    plt.savefig(f"build8/anim_{i//25:03d}.png")

    # Close the figure to free memory
    plt.close(fig)

In [52]:
# !ffmpeg -i build6/anim_%03d.png -r 8 -vframes 600 -y -pix_fmt yuv420p build6/anim.mp4 
!ffmpeg -i build8/anim_%03d.png -r 5 -vframes 300 -y -pix_fmt yuv420p -filter:v "setpts=2.7*PTS" build8/anim3.mp4


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e